# Desafio de NLP - NeuralMed
- **CANDIDATO:** Pedro Luís Azevedo Costa

# Importações

In [43]:
import pandas as pd
import magic
import unicodedata
from bs4 import BeautifulSoup
from striprtf.striprtf import rtf_to_text
import re

from IPython.display import HTML, display

import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import random

from spacy.training.example import Example
from sklearn.metrics import precision_score, recall_score, f1_score




# Etapa 1 
- Faça uma análise descritiva em cima dos documentos. O que você conclui apenas olhando para eles? Qual a forma de escrita dos documentos? Há tipos diferentes? Olhando os textos consegue identificar alguma alteração comum entre os pacientes? 

## Carregando o dataset

In [44]:
file_path = "sample_teste_nlp.csv"
df_doc = pd.read_csv(file_path)
df_doc.head()

,patient_id,age,sex,ehr_date,text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...


## Análise Exploratória  

O dataset contém 5 colunas:  
- `patient_id`: numérica  
- `age`: numérica  
- `sex`: categórica  
- `ehr_date`: datetime  
- `text`: textual  

Conforme mencionado no enunciado, o foco deste exercício está na coluna `text`, que contém os documentos a serem analisados. Ao observar as primeiras 5 linhas do dataset, é possível identificar três tipos diferentes de formatação de texto: **plain text**, **HTML** e **RTF**.  

Assim, o primeiro passo desta análise preliminar é descobrir todos os tipos possíveis de formatação.  


In [45]:
def detect_mime_type(text):
    """
    Detecta a formatação do texto
    """
    mime = magic.Magic(mime=True)
    return mime.from_buffer(text.encode('utf-8'))

# Aplica a função detect_mime_type na coluna 'text' do dataframe
df_doc['text_type'] = df_doc['text'].apply(lambda x: detect_mime_type(x))

tipos_texto = df_doc['text_type'].unique()

print("Tipos de texto encontrados: ", tipos_texto)

# Quantidade de textos por tipo
count_text_type = df_doc['text_type'].value_counts()

print("Quantidade de textos por tipo: ", count_text_type)


Tipos de texto encontrados:  ['text/html' 'text/rtf' 'text/plain']
Quantidade de textos por tipo:  text_type
text/html     4253
text/rtf      1852
text/plain     729
Name: count, dtype: int64


Conforme mostrado na célula acima, identificamos três tipos de texto, com predominância de **HTML**, que será o foco da análise.  

Embora o ideal fosse realizar uma avaliação cuidadosa de cada tipo de texto, devido às restrições de tempo e recursos, concentraremos nossos esforços no formato **HTML**. Esse formato foi escolhido por sua maior predominância e pela estrutura mais simples de navegar em comparação aos demais.  


## Visualizando Exemplos

In [46]:
# Avaliando alguns textos em HTML

df_filtered_html = df_doc.loc[df_doc['text_type'] == 'text/html']

display(HTML(df_filtered_html['text'].iloc[0]))
display(HTML(df_filtered_html['text'].iloc[100]))
display(HTML(df_filtered_html['text'].iloc[200]))
display(HTML(df_filtered_html['text'].iloc[300]))
display(HTML(df_filtered_html['text'].iloc[400]))
display(HTML(df_filtered_html['text'].iloc[500]))




## Extraindo Padrões

Ao observar os documentos acima, é possível notar que alguns possuem um formato que facilita a anotação de diagnósticos. Há um campo chamado **"Hipótese Diagnóstica"** e, logo após, uma sigla ou o nome completo do diagnóstico.  

Para este trabalho, vamos nos focar nesse atributo.  

Deste modo, o próximo passo é definir uma função capaz de extrair estes rótulos.

In [47]:
df_filtered_html['text'].iloc[400]

'<html tasy="html5"><body><p style="text-align:left;"><span style="font-style: normal; font-variant: normal; font-stretch: normal; font-size: 11pt; font-family: Arial; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255);">\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0<strong>\xa0 ANAMNESE - CENTRO CLÍNICO</strong></span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">Queixa Principal: TUMORAÇÃO 2º DEDO PÉ DIR HA TEMPOS</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;">Alergias: Nega Alergia à Medicamentos</p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">HDA:\xa0</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">HPP:\xa0</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style=""><span style="">Sinais Vitais</span></span></p><p

In [48]:
def extrair_hipotese_diagnostica(html):
    # Parseia o HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Encontra o texto que vem após 'Hipótese Diagnóstica:'
    texto = soup.get_text()

    # Usando regex para encontrar o texto após 'Hipótese Diagnóstica:'
    padrao = re.compile(r'Hipótese Diagnóstica:([^\n]+)')
    resultado = padrao.search(texto)

    if resultado:
        match_result = resultado.group(1).strip()
        match_result = match_result.replace('\xa0', '||')
        match_result = match_result.replace('\u200b', '||')
        match_result = match_result.split('||')[0]

        return match_result  # Retorna o texto após 'Hipótese Diagnóstica:'
    else:
        return None  # Caso não encontre o padrão

extrair_hipotese_diagnostica(df_filtered_html['text'].iloc[5])

In [49]:
df_doc['hipotese_diagnostica'] = df_doc['text'].apply(lambda x: extrair_hipotese_diagnostica(x))
# Lower case para padronizar o texto
df_doc['hipotese_diagnostica'] = df_doc['hipotese_diagnostica'].str.lower()
df_doc.head()

/tmp/ipykernel_866699/2961999388.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


,patient_id,age,sex,ehr_date,text,text_type,hipotese_diagnostica
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",text/html,hernia inguinal
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:...",text/html,None
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,text/rtf,None
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D,text/plain,None
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,text/rtf,None


In [50]:
# Calcula o número total de hipóteses diagnósticas extraídas
total_hipoteses = df_doc['hipotese_diagnostica'].count()

# Calcula o número de hipóteses distintas extraídas|
hipoteses_distintas = df_doc['hipotese_diagnostica'].unique().shape[0]

print("Total de hipóteses diagnósticas extraídas: ", total_hipoteses)

print("Total de hipóteses diagnósticas distintas extraídas: ", hipoteses_distintas)

Total de hipóteses diagnósticas extraídas:  1103
Total de hipóteses diagnósticas distintas extraídas:  753


In [9]:
df_doc['hipotese_diagnostica'].dropna().value_counts().head(20)

hipotese_diagnostica
                                                 45
conduta:                                         40
rotina                                           18
ivas                                             16
has                                              14
r10                                              12
geca                                             12
z10                                              11
r51                                               9
amigdalite                                        9
itu                                               8
gastroenterite                                    6
dispepsia                                         6
sinusite                                          6
dor abdominal                                     5
lombalgia                                         5
z370 - nascimento unico, nativivo [nado-vivo]     5
enxaqueca                                         5
dorsalgia                                  

Ao analisar as 20 hipóteses diagnósticas mais frequentes, é possível identificar alguns erros de extração, como a presença de um campo vazio na primeira posição e a ocorrência do termo "conduta:". Além disso, observa-se que "rotina" aparece como uma hipótese diagnóstica, o que não é apropriado. Portanto, a melhor abordagem para lidar com esses casos é remover as linhas que contêm essas classificações.

In [56]:
df_doc[~df_doc['hipotese_diagnostica'].str.contains('conduta', na=False)]

,patient_id,age,sex,ehr_date,text,text_type,hipotese_diagnostica
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",text/html,hernia inguinal
6,3811139,41.0,F,2023-11-01 16:10:17,"<html tasy=""html5""><body><p style=""text-align:...",text/html,dm2
7,4648024,52.0,M,2024-02-01 19:38:11,"<html tasy=""html5""><body><p style=""text-align:...",text/html,cefaleia
10,2426382,20.0,F,2023-09-05 10:14:14,"<html tasy=""html5""><body><p style=""text-align:...",text/html,k35
15,3037495,4.0,F,2023-11-12 10:05:06,"<html tasy=""html5""><body><p><span style=""""><sp...",text/html,j20 + j18
...,...,...,...,...,...,...,...
6794,3917585,21.0,M,2024-03-06 23:16:59,"<html tasy=""html5""><body><p style=""text-align:...",text/html,trauma
6797,5000919,28.0,F,2023-10-10 16:57:12,"<html tasy=""html5""><body><p style=""text-align:...",text/html,z108
6800,1071389,83.0,F,2023-09-26 21:05:48,"<html tasy=""html5""><body><p style=""text-align:...",text/html,r10
6813,4894228,28.0,F,2023-10-05 12:26:28,"<html tasy=""html5""><body><p style=""text-align:...",text/html,dengue ?


In [57]:
# Remove linhas com hipose_diagnostica nula, contém "conduta" e "rotina"
df_doc = df_doc[~(df_doc['hipotese_diagnostica'] == '')]
df_doc = df_doc[~df_doc['hipotese_diagnostica'].str.contains('conduta', na=False)]
df_doc = df_doc[~df_doc['hipotese_diagnostica'].str.contains('rotina', na=False)]

# Calcula o número total de hipóteses diagnósticas extraídas
total_hipoteses = df_doc['hipotese_diagnostica'].count()

# Calcula o número de hipóteses distintas extraídas|
hipoteses_distintas = df_doc['hipotese_diagnostica'].unique().shape[0]

print("Total de hipóteses diagnósticas extraídas: ", total_hipoteses)

print("Total de hipóteses diagnósticas distintas extraídas: ", hipoteses_distintas)

display(df_doc['hipotese_diagnostica'].dropna().value_counts().head(20))

Total de hipóteses diagnósticas extraídas:  862
Total de hipóteses diagnósticas distintas extraídas:  617


hipotese_diagnostica
ivas                                             16
has                                              14
r10                                              12
geca                                             12
z10                                              11
amigdalite                                        9
r51                                               9
itu                                               8
dispepsia                                         6
sinusite                                          6
gastroenterite                                    6
z370 - nascimento unico, nativivo [nado-vivo]     5
lombalgia                                         5
dorsalgia                                         5
enxaqueca                                         5
dor abdominal                                     5
j159 - pneumonia bacteriana nao especificada      4
alergia                                           4
amigdalite aguda                           

Com base na tabela acima, conclui-se que os diagnósticos mais comuns nos documentos analisados são **IVAS** e **HAS**, siglas que correspondem a "Infecções Agudas das Vias Aéreas Superiores" e "Hipertensão Arterial Sistêmica".


# Etapa 2
- Como você extrairia informações de medicamentos, exames e doenças do paciente? Analise os dados que tem em mãos e descreva o pipeline de desenvolvimento para um modelo como esses, levando em consideração todo o processo de análise, anotação, treino até a validação do modelo. 

A etapa anterior abordou a extração de hipóteses diagnósticas. Nesta etapa, o foco será preparar o conjunto de dados no formato necessário para treinar um modelo no spaCy. Além disso, o conjunto de dados será dividido em três partes: treino, validação e teste, garantindo uma estrutura adequada para a avaliação do modelo.

O primeiro passo para prepara o conjunto de dados é extrair o texto do formato html de modo a convertê-lo em plain text

In [58]:
# Função que limpa todo o html e deixa apenas o plain text
def clean_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator = " ")
    #text = re.sub(r'\u200b', ' ', text)
    return text


# Função que limpa caracteres especiais como \xa0 e \u200b
def clean_special_characters(texto):
    # Normaliza o texto para remover caracteres compostos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    # Remove caracteres de controle e espaços invisíveis
    texto_limpo = ''.join(c for c in texto_normalizado if c.isprintable())
    return texto_limpo.strip()

# Faz o map da função clean_html para a coluna 'text' do dataframe
def map_clean_text_type(row):
    if row['text_type'] == 'text/html':
        clean_text = clean_html(row['text'])
        clean_text = clean_special_characters(clean_text)
        return clean_text.lower()
    else:
        return row['text'].lower()

# Filtra apenas textos do tipo 'text/html'
df_doc_filtered = df_doc.loc[df_doc['text_type'] == 'text/html']
# Aplica a função map_clean_text_type
df_doc_filtered['processed_text'] = df_doc_filtered.apply(map_clean_text_type, axis=1)
# Exibe as colunas 'processed_text' e 'text'
df_doc_filtered[['processed_text', 'text']].head()


,processed_text,text
0,anamnese - centro clínico queixa principal:...,"<html tasy=""html5""><body><p style=""text-align:..."
6,anamnese - centro clínico queixa principal:...,"<html tasy=""html5""><body><p style=""text-align:..."
7,anamnese/exame físico - ps anamnese: queixa...,"<html tasy=""html5""><body><p style=""text-align:..."
10,evolução de enfermagem 6 o dih hipótes...,"<html tasy=""html5""><body><p style=""text-align:..."
15,anamnese/exame físico - ps queixa principal:...,"<html tasy=""html5""><body><p><span style=""""><sp..."


In [ ]:
# Função para transformar o DataFrame em um formato SpaCy treinável
def create_training_data(df):
    training_data = []
    for text, entity in zip(df["processed_text"], df["hipotese_diagnostica"]):
        start = text.find(entity)
        if start != -1:  # Certifica de que a entidade existe no texto
            end = start + len(entity)
            training_data.append((text, {"entities": [(start, end, "HIPOTESE_DIAGNOSTICA")]}))
    return training_data

In [ ]:
# Criar os dados de treinamento no formato SpaCy
training_data = create_training_data(df_doc_filtered)

# Separa em treino (80%), validação (10%) e teste (10%)
random.shuffle(training_data)
train_data = training_data[:int(len(training_data) * 0.8)]
valid_data = training_data[int(len(training_data) * 0.8):int(len(training_data) * 0.9)]
test_data = training_data[int(len(training_data) * 0.9):]

In [ ]:
# Função para salvar os dados em formato binário do SpaCy
def save_training_data(training_data, output_path="training_data.spacy"):
    nlp = spacy.blank("pt")  # Usando modelo base em português
    db = DocBin()  # Objeto para armazenar dados
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents  # Adiciona as entidades no Doc
        db.add(doc)
    db.to_disk(output_path)
    print(f"Dados de treinamento salvos em {output_path}")

# Salvar os conjuntos de treino, validação e teste no formato binário do SpaCy
save_training_data(train_data, "train_data.spacy")
save_training_data(valid_data, "val_data.spacy")
save_training_data(test_data, "test_data.spacy")


Dados de treinamento salvos em train_data.spacy
Dados de treinamento salvos em val_data.spacy
Dados de treinamento salvos em test_data.spacy


## Treinamento

O treinamento será realizado utilizando o módulo "train" nativo do spaCy, ativado via linha de comando com a especificação de um arquivo de configuração no formato `.cfg`. Esse arquivo contém as definições dos hiperparâmetros e outras configurações necessárias para o treinamento do modelo. Abaixo estão algumas configurações relevantes selecionadas e suas justificativas:

- **batch_size = 32**: Foi escolhido um tamanho de batch pequeno para facilitar a convergência em um conjunto de dados com menos de 1000 sentenças. Isso também melhora a estabilidade do treinamento inicial.
  
- **dropout = 0.3**: Um valor de dropout relativamente elevado visa reduzir o risco de overfitting, que é um desafio comum ao trabalhar com conjuntos de dados limitados.

- **learn_rate = 0.0005**: Uma taxa de aprendizado pequena permite ajustes mais finos nos pesos do modelo, evitando oscilações em cenários de treinamento com poucos dados.

- **max_epochs = 20**: Um limite de épocas reduzido para garantir que o treinamento não exceda o necessário, prevenindo overfitting e otimizando o uso de recursos computacionais.

- **patience = 1000**: O critério de early stopping foi configurado para aguardar até 1000 iterações sem melhorias no desempenho antes de interromper o treinamento. Essa configuração busca garantir que o modelo tenha tempo suficiente para explorar possíveis melhorias, mesmo em um conjunto de dados pequeno.

- **eval_frequency = 50**: Configurado para realizar avaliações frequentes durante o treinamento, possibilitando um monitoramento mais próximo do progresso e da estabilidade do modelo.

- **grad_clip = 0.5**: O clipping do gradiente foi ajustado para limitar grandes variações nas atualizações dos pesos, promovendo maior estabilidade no processo de aprendizado.

Essas escolhas foram feitas com o objetivo de adaptar o treinamento ao tamanho do conjunto de dados, reduzindo o risco de overfitting e melhorando a eficiência do processo. 

Outro ponto importante a se mencionar é que a métrica escolhida para ser maximizada ao longo do processo de treinamento foi o F1-Score. Essa escolha é motivada pela natureza do problema de NER, no qual é importante não apenas identificar corretamente as entidades (alta precisão), mas também capturar todas as entidades relevantes (alto recall). O F1-Score fornece um equilíbrio entre essas métricas, sendo particularmente adequado para avaliar o desempenho em tarefas onde tanto falsos positivos quanto falsos negativos podem ter impacto significativo. Isso garante uma avaliação mais abrangente e justa do modelo, especialmente em conjuntos de dados pequenos e desbalanceados.

In [ ]:
# Linha de comando para treinar o modelo
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./val_data.spacy 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-01-18 18:52:50,015] [WARNING] [W112] The model specified to use for initial vectors (pt_core_news_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0005
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.33    0.00    0.00    0.00    0.00
  0      50          1.01   1343.37    0.00    0.00    0.00    0.00
  0     100          0.52     69.43   49.68   51.32   48.15    0.50
  0     150          0.95     56.29   50.93   51.25   50.62    0.51
  0     200          3.36     93.15   61.54   59.09   64.20    0.62
  0     250          1.46     61.41   56.47   53.93   59.26    0.56
  0   

# Avaliação

O melhor desempenho foi atingido na época 2, passo 1450, onde foi atingido um F1-Score de 0.88 no conunto de validação. Abaixo, será realizada a avaliação no conunto de teste a partir do módulo "evalute" do spacy.

In [35]:
!python -m spacy evaluate output/model-best test_data.spacy --output metrics.json

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   81.82 
NER R   77.78 
NER F   79.75 
SPEED   3138  


=============================== NER (per type) ===============================

                           P       R       F
HIPOTESE_DIAGNOSTICA   81.82   77.78   79.75

✔ Saved results to metrics.json


O F1-Score obtido no conjunto de teste foi de aproximadamente 0.8, que configura um valor relativamente alto, mas vale notar que muito provavelmente este modelo não se generaliza para textos com escritas diferentes dos usados no conjunto de treinamento. Isso se dá porque, por questões de tempo e simplificação, utilizou-se apenas a anotaçõa de dados através da extração das palavras que se encontram logo após o campo "Hipótese Diagnóstica". Abaixo será realizado um pequeno teste para validar esta hipótese.

In [59]:
# Carrega o modelo treinado
nlp = spacy.load("./output/model-best")

In [ ]:
# Faz a predição em um texto aleatório do conjunto de teste
sentenca = test_data[2][0]

print("Sentença: ", sentenca)

doc = nlp(sentenca)

# Exibe as entidades preditas
print("Entidades preditas: ")
if doc.ents:
    for ent in doc.ents:
        print(ent.text, ent.label_)
else:
    print("Nenhuma entidade encontrada.")

Sentença:  anamnese/exame físico - ps anamnese:   queixa e duração:  febre há 2 dias + dor abdominal + vomitos 4 vezes hoje alergias:  d esconhece antecedentes pessoais:  ( ) nenhum ap ( ) cardiopatia ( ) hipertens ã o ( ) dm ( ) atualmente gr á vida medicamento de uso: sinais vitais/ controle:  pa:   x   / fc:   / sato2:   / fr:   / temperatura:   / dor:   / glicemia capilar :    /  peso:   / altura:    exame físico:  beg, lote, chaaa 1 - snc:  glasgow 15 sem d é ficit motor. pupilas isoc ó ricas e fotorreagentes, nuca livre, sem sinais meningismo.  2 - c e p: hiperemia de of 3 - acv:  bulhas r í tmicas normofon é ticas, sem sopros aud í veis. 4 - ar:  mu rmúrio vesi cular (+) bilateral, sem ru í dos advent í cios. 5 - abd:  plano, ruídos hidroaéreos (+), fláci do, indolor à palpação,  sem visceromegalias, sem sinais de irrita çã o peritoneal (db -). 6 - membros:  perfus ã o preservada, panturrilhas livres, sem edemas.  hipótese diagnóstica:   faringite  cond

O modelo funcionou corretamente ao extrair "faringite" de uma sentença do conjunto de teste, à qual não foi exposto durante seu treinamento. Para avaliar a generalização do modelo, vamos fazer este mesmo teste, com uma frase inventada por mim, com a mesma entidade (faringite), mas num texto fora do padrão de treino.

In [67]:
sentenca = "O paciente se encontra com faringite aguda."

print("Sentença: ", sentenca)

doc = nlp(sentenca)

# Exibe as entidades preditas
print("Entidades preditas: ")
if doc.ents:
    for ent in doc.ents:
        print(ent.text, ent.label_)
else:
    print("Nenhuma entidade encontrada.")

Sentença:  O paciente se encontra com faringite aguda.
Entidades preditas: 
Nenhuma entidade encontrada.


O modelo não conseguiu identificar corretamente a entidade "faringite" nesta sentença, evidenciando um viés significativo em relação ao formato de escrita do documento. Além disso, o F1-Score obtido reflete esse viés, já que o conjunto de teste segue o mesmo padrão de escrita que o conjunto de treinamento.

Em um cenário de aplicação real, seria essencial investir mais esforços na anotação e preparação do conjunto de dados para mitigar esses problemas. Soluções baseadas apenas em expressões regulares, como as utilizadas neste caso, frequentemente não são suficientes para treinar modelos de alto desempenho. Duas abordagens mais robustas para lidar com esse desafio incluem:

1. **Pré-anotação de dados com LLMs**: Utilizar modelos de linguagem para realizar uma pré-anotação automática, seguida de curadoria manual nas labels com pior desempenho. Ferramentas como o Amazon A2I podem ser úteis nesse processo.  
2. **Serviços especializados de anotação**: Contratar serviços de anotação oferecidos por terceiros, como o Amazon Ground Truth, para garantir um conjunto de dados mais variado e representativo.

Ressalto que meu background em serviços da AWS me faz lembrar dessas soluções específicas, mas alternativas semelhantes estão disponíveis em outras plataformas de nuvem e podem ser igualmente adequadas ao contexto.


# Etapa 3
- Se fizesse as extrações via prompt, que tipo de prompt você usaria para extrair informações desses documentos médicos? Mostre exemplos.